In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import time
import os
from os import path
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD

In [ ]:
movie = pd.read_csv('/content/movies.csv', sep=',',names=['MovieID','Title','Genres'],encoding='latin-1')
rating = pd.read_csv('ratings.csv',sep=',', names=['UserID','MovieID','Rating','Timestamp'])
user = pd.read_csv('users.csv',sep=',',names=['Gender','Age','Occupation','Zip-code'])

In [ ]:
R_df = rating.pivot_table(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,3.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
R = R_df.to_numpy()                                               #Convert into an array
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)         

In [ ]:
original_df = pd.DataFrame(R_demeaned,columns=R_df.columns)
original_df.head(7)

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.932193,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,...,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807,-0.067807
1,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,0.853696,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,...,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304,-0.146304
2,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,...,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782,-0.060782
3,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,...,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878,-0.026878
4,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,1.809713,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,2.809713,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,0.809713,-0.190287,-0.190287,-0.190287,-0.190287,4.809713,-0.190287,-0.190287,3.809713,3.809713,-0.190287,2.809713,-0.190287,-0.190287,2.809713,-0.190287,3.809713,...,-0.190287,-0.190287,-0.190287,-0.190287,-0.190287,-0.19028

In [ ]:
#Initiate SVD with numpy 
U, sigma, Vt= np.linalg.svd(R_demeaned, full_matrices=False)

In [ ]:
print(U)
print(U.shape)

[[-0.01779596  0.00655866  0.00399804 ... -0.00656571  0.02076927
  -0.00496797]
 [-0.03458528  0.00069955 -0.00334867 ... -0.0119925   0.01311718
   0.00170266]
 [-0.0204113  -0.00719057  0.01042589 ... -0.04046498  0.03250324
  -0.03312821]
 ...
 [-0.00947689 -0.00278895 -0.0261903  ... -0.22553872 -0.20411384
   0.06264834]
 [-0.07320281 -0.04924934 -0.08093162 ... -0.00156582  0.00783792
   0.01167202]
 [-0.00843601  0.0303729   0.00679459 ...  0.01669114 -0.004329
   0.00196828]]
(677, 677)


In [ ]:
print(sigma)
print(sigma.shape)

[483.07168855 215.91261879 187.7615078  168.45265969 151.34494263
 145.84702155 128.18614233 120.83634803 112.00477618 111.22840532
 104.98521574  99.6102794   96.77058911  94.28812165  92.81803999
  91.09814484  90.30730234  88.56956641  86.90466281  85.31861017
  84.36994333  83.48089174  82.86582354  80.80276381  79.88831451
  78.60988799  77.8778239   77.55351558  76.67041452  76.09887048
  75.1290057   74.37369615  73.99062937  73.53128554  72.93661616
  72.13429196  71.51924111  71.17956959  70.97594319  70.80265562
  70.24553171  69.67446657  68.98506484  68.60151976  68.27182597
  67.47689931  67.35467694  67.16194845  66.87908009  66.25236484
  66.22474412  65.69580313  65.29116546  65.13053365  64.61443918
  64.25421093  64.14502452  63.71559114  63.14333598  62.82733471
  62.6064317   62.21659098  61.87098071  61.72629154  61.58577713
  61.30942483  61.10632244  61.0089521   60.77296229  60.54480111
  60.47703028  59.9600489   59.85448457  59.50437696  59.34175193
  58.97171

In [ ]:
print(Vt)
print(Vt.shape)

[[-0.07489579 -0.01799129 -0.00940692 ...  0.00793257  0.0102336
  -0.00269577]
 [-0.01600688 -0.0273547  -0.01184273 ...  0.00496119  0.00390009
   0.00136027]
 [-0.01651871  0.01531399 -0.01462607 ... -0.00195168  0.0025888
  -0.02008595]
 ...
 [ 0.01781447  0.01993253  0.03525913 ... -0.01134804 -0.00092717
  -0.02566639]
 [-0.01311838 -0.00216505  0.01743193 ... -0.00974598 -0.00317292
  -0.01830849]
 [-0.01118345 -0.02288513  0.02178908 ...  0.00390652  0.00080175
   0.04348245]]
(677, 3274)


In [ ]:
sigma = np.diag(sigma)
sigma

In [ ]:
x = np.tile(user_ratings_mean, (R_demeaned.shape[0],1))

In [ ]:
x

In [ ]:
k =10
s=s[0:k,0:k]
U=U[:,0:k]
V=V[0:k,:]

In [ ]:
Usk=np.dot(U,sigma)
skV=np.dot(sigma,Vt)
UsV = np.dot(Usk, skV)


In [ ]:
all_user_predicted_ratings

In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [ ]:
preds_df

In [ ]:
preds_df.iloc[3].sort_values(ascending=False)

In [ ]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations